In [10]:
import requests
from invisibleroads_macros.disk import make_folder
from os.path import expanduser, join

def download(target_path, source_url):
    response = requests.get(source_url)
    open(target_path, 'w').write(response.content)
    return target_path

target_folder = make_folder(expanduser('~/Experiments/spatiotemporal'))

In [11]:
geojson_path = download(
    join(target_folder, 'nyc-traffic-injuries.json'), 
    'http://www.nyc.gov/html/dot/downloads/misc/injury_all_monthly.json')
geojson_path

'/home/rhh/Experiments/spatiotemporal/nyc-traffic-injuries.json'

In [12]:
shapefile_path = download(
    join(target_folder, 'nyc-traffic-injuries.shp.zip'),
    'http://www.nyc.gov/html/dot/downloads/misc/injury_all_monthly_shapefile.zip')
shapefile_path

'/home/rhh/Experiments/spatiotemporal/nyc-traffic-injuries.shp.zip'

In [13]:
# Get first record
import fiona

In [15]:
geojson_collection = fiona.open(geojson_path)

In [17]:
geojson_collection.bounds

(-74.2539230306024, 40.49947769792743, -73.70059800086655, 40.91246913562538)

In [18]:
geojson_collection[0]

{'geometry': {'coordinates': (-73.7917447266822, 40.72578884918672),
  'type': 'Point'},
 'id': '0',
 'properties': OrderedDict([(u'MVOInjurie', 1),
              (u'MN', u'1'),
              (u'Injuries', 2),
              (u'BikeInjuri', 0),
              (u'YR', u'2009'),
              (u'PedInjurie', 1)]),
 'type': 'Feature'}

In [19]:
geojson_collection[0]['properties']

OrderedDict([(u'MVOInjurie', 1),
             (u'MN', u'1'),
             (u'Injuries', 2),
             (u'BikeInjuri', 0),
             (u'YR', u'2009'),
             (u'PedInjurie', 1)])

In [22]:
shapefile_path

'/home/rhh/Experiments/spatiotemporal/nyc-traffic-injuries.shp.zip'

In [25]:
shapefile_collection = fiona.open('/', vfs='zip://' + shapefile_path)

In [26]:
shapefile_collection.bounds

(-74.2539230306024, 40.49947769792743, -73.70059800086655, 40.91246913562538)

In [29]:
shapefile_collection.crs_wkt

u'GEOGCS["GCS_WGS_1984",DATUM["WGS_1984",SPHEROID["WGS_84",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]'

In [30]:
shapefile_collection.crs

{'init': u'epsg:4326'}

In [32]:
geojson_collection.crs

{'init': u'epsg:4326'}

In [33]:
geojson_collection.crs_wkt

u'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]]'

In [34]:
shapefile_collection[0]

{'geometry': {'coordinates': (-73.7917447266822, 40.72578884918672),
  'type': 'Point'},
 'id': '0',
 'properties': OrderedDict([(u'Injuries', 2),
              (u'PedInjurie', 1),
              (u'BikeInjuri', 0),
              (u'MVOInjurie', 1),
              (u'MN', u'1'),
              (u'YR', u'2009')]),
 'type': 'Feature'}